# Create edge shapefiles

- Input: node shapefiles 
- Output: edge shapefiles
- Time cost: the script takes 1~2 hours (?) to complete all the iterations.

The edge shapefiles are used for visualizing networks, not in the modeling process.


In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import networkx as nx
import matplotlib.pyplot as plt
import pickle
import copy
import scipy.sparse as sp
from scipy.sparse import csr_matrix
from sklearn.linear_model import LinearRegression
from shapely.geometry import LineString


Bad key text.latex.unicode in file /home/jtl/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 112 ('text.latex.unicode : False # use "ucs" and "inputenc" LaTeX packages for handling')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key text.latex.preview in file /home/jtl/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 125 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file /home/jtl/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 157 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append("../")
import utils

In [4]:
import importlib
importlib.reload(utils)

<module 'utils' from '../utils.py'>

In [5]:
# # read files
# with open("../../data/02_intermediate/greater_boston_ct_socioecon_shp.pickle", 'rb') as f:
#     df_shp = pickle.load(f)


In [6]:
# read files
with open("../../data/02_intermediate/boston_miami_chicago_nyc_ct_shp_dic.pickle", 'rb') as f:
    shp_dic = pickle.load(f)


In [7]:
shp_dic.keys()

dict_keys(['boston', 'miami', 'chicago', 'nyc', 'total'])

In [8]:
# create edge_dic
# this line takes 1~2 hours.
edge_dic = {}
city_list = ['boston', 'chicago', 'miami', 'nyc']
origin_col_name = 'GEOID_home'
destination_col_name = 'GEOID'
for city in city_list:
    print(city)
    edge_dic[city]=utils.turn_node_to_edge_shp(shp_dic[city], origin_col_name, destination_col_name)


boston
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000


KeyboardInterrupt: 

In [9]:
# add distance and change index names for the three edge shapefiles.
for city in edge_dic.keys():
    edge_dic[city]['distance'] = edge_dic[city].geometry.length
    edge_dic[city].index = list(tuple(zip(edge_dic[city].GEOID_home, edge_dic[city].GEOID)))
    

/home/jtl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
# # Turn the node to edge shapefile
# origin_col_name = 'GEOID_home'
# destination_col_name = 'GEOID'
# edge_shp = utils.turn_node_to_edge_shp(df_shp, origin_col_name, destination_col_name)


In [17]:
# edge_shp['distance'] = edge_shp.geometry.length

/home/jtl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  """Entry point for launching an IPython kernel.


In [27]:
# edge_shp.index = list(tuple(zip(edge_shp.GEOID_home, edge_shp.GEOID)))

In [12]:
edge_dic['nyc']

,GEOID_home,GEOID,geometry,distance
"(34003001000, 34003001000)",34003001000,34003001000,"LINESTRING (-74.13330 41.03321, -74.13330 41.0...",0.000000
"(34003001000, 34003002100)",34003001000,34003002100,"LINESTRING (-74.13330 41.03321, -73.91991 40.9...",0.225072
"(34003001000, 34003002200)",34003001000,34003002200,"LINESTRING (-74.13330 41.03321, -73.94255 41.0...",0.192566
"(34003001000, 34003002300)",34003001000,34003002300,"LINESTRING (-74.13330 41.03321, -73.95099 40.9...",0.186620
"(34003001000, 34003003100)",34003001000,34003003100,"LINESTRING (-74.13330 41.03321, -73.98664 40.9...",0.179064
...,...,...,...,...
"(36119985000, 36119015000)",36119985000,36119015000,"LINESTRING (-73.70025 41.04635, -73.60426 41.3...",0.302281
"(36119985000, 36119981000)",36119985000,36119981000,"LINESTRING (-73.70025 41.04635, -73.81050 41.0...",0.116597
"(36119985000, 36119982000)",36119985000,36119982000,"LINESTRING (-73.70025 41.04635, -73.86849 41.1...",0.198312
"(36119985000, 36119984000)",36119985000,36119984000,"LINESTRING (-73.70025 41.04635, -73.93620 41.2...",0.299909


# Save

In [13]:
# save
with open('../../data/02_intermediate/edge_shp_dic.pickle', 'wb') as f:
    pickle.dump(edge_dic, f)
